# Runnung BIAFLOWS workflows locally

## Introduction

BIAFLOWS allows to package and run image analysis in a reproducible way. This notebook demonstrates how to run BIAFLOWS image analysis workflows on your own images on your local machine. 

You need to have Python [Anaconda distribution](https://www.anaconda.com/distribution/#download-section) and [Docker](https://docs.docker.com/install/) installed on your machine. Additionally, the Docker-daemon should be running. 

## Available workflows

The two cells below will display a list of the available workflows. You can find out more about BIAFLOWS at:
* [BIAFLOWS web-application](https://biaflows.neubias.org/) provides annotated images and workflows and it can benchmark the workflows online
* BIAFLOWS workflows are hosted in [neubiaswg5 organization on Dockerhub](https://hub.docker.com/search?q=neubiaswg5&type=image)
* BIAFLOWS workflows source code and configuration files are on [GitHub](https://github.com/orgs/Neubias-WG5/dashboard). 

In [ ]:
%%capture biaflowsImages --no-stderr
!docker search --no-trunc neubiaswg5 | grep w_
lines = str(biaflowsImages).split("\r\n")

In [ ]:
import pandas as pd
import numpy as np
workflows=[]
descriptions=[]
for line in lines:
    if line=='': 
        continue
    parts = line.split("  ")
    parts = list(filter(lambda a: a != '', parts))
    workflows.append(parts[0].split('/')[1])
    description = parts[1].lstrip()
    if (description=='0'):
        description = ''
    descriptions.append(description)

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
df = pd.DataFrame(zip(workflows,descriptions), columns=["workflow", "description"])
df.index = np.arange(1, len(df)+1)
display(df)

## Running a workflow

BIAFLOWS workflows are Docker images and they can be run with ``docker run`` command + workflow Docker image name (as on DockerHub).
You need to provide a local folder with two subfolders: **in** (input images) and **out** (results).
You can either choose an existing folder or execute the following cell to create the two subfolders.

In [ ]:
!mkdir ./data
!mkdir ./data/in
!mkdir ./data/out

In this example we will execute the workflow ``w_nucleisegmentation-imagej`` for 2D nuclei segementation. If you have no compatible images, you can download example images by executing the following cell.

In [ ]:
inFolder = "./data/in"
!cd $inFolder && wget -np -nd -r -l1 -A_Nuc_Sat.tif https://zenodo.org/record/3613710/
!cd $inFolder && wget -np -nd -r -l1 -A_Nuc_Norm.tif https://zenodo.org/record/3613710/

The workflow has two parameters:
* the radius for the Laplacian filter ``ij_radius``
* the segmentation threshold ``ij_threshold``
You might need to adapt the parameter values to your images.

The default parameters of a workflow can be found in the [BIAFLOWS web-application](https://biaflows.neubias.org/) or in the [descriptor.json](https://github.com/Neubias-WG5/W_NucleiSegmentation-ImageJ/blob/master/descriptor.json) file of the workflow on github.

The option ``-nmc`` means ``no metrics calculation`` and the switch ``-local`` means that no download from or upload to the BIAFLOWS server is performed (images, annotations, metrics).

In [ ]:
inFolder = "./data/in"
outFolder = "./data/out"
!docker run -v $(pwd)/data:/data -it neubiaswg5/w_nucleisegmentation-imagej:1.12.3 \
    --ij_radius 5 \
    --ij_threshold -0.5 \
    --infolder /data/in \
    --outfolder /data/out \
    -nmc \
    --local

## Displaying the results

### Loading the input and result images

In [ ]:
import os
import tifffile
files = [i.path for i in os.scandir(inFolder) if i.is_file()]
in_paths_sources = [i for i in files if ('.tif' in i) ]
sources = []
for path in in_paths_sources:
    source = tifffile.imread(path)
    sources.append(source)

outfiles = [i.path for i in os.scandir(outFolder) if i.is_file()]
out_paths_sources = [i for i in outfiles if ('.tif' in i) ]
outsources = []
for path in out_paths_sources:
    outsource = tifffile.imread(path)
    outsources.append(outsource)

### Computing number of nuelcei (per image) from the result masks

In [ ]:
import numpy as np
counts = []
for image in outsources:
    counts.append(len(set(image.flatten()))-1)

### Displaying input images, result masks and nuclei counts

In [ ]:
import matplotlib.pyplot as plt
for image, result, path, count in zip(sources, outsources, in_paths_sources, counts):

    # display results
    fig, axes = plt.subplots(1, 2, figsize=(8, 3), sharex=True, sharey=True)
    ax = axes.ravel()
    ax[0].imshow(image, cmap=plt.cm.gray)
    ax[0].axis('off')
    ax[0].set_title('Original')

    ax[1].imshow(result, cmap=plt.cm.prism)
    ax[1].set_title('Indexed Mask')

    fig.tight_layout()

    plt.show()
    
    print(path + " - nr of cells: ", count)